# 🚢 Titanic Survival Prediction: A Classification Baseline

## 1. Data Acquisition and Initial Inspection
*Goal: Load data and check its quality (missing values, types).*

### 1.1 Load DataFrames

In [3]:
import pandas as pd
import os

In [4]:
# Define the path to raw data folder
data_dir = os.path.join(os.getcwd(), '..', 'data', 'raw')
train_file = os.path.join(data_dir, 'train.csv')
test_file = os.path.join(data_dir, 'test.csv')

# Load the Dataframes
train_df = pd.read_csv(train_file)
test_df = pd.read_csv(test_file)

print("Train Data Shape:", train_df.shape)
print("Test Data Shape:", test_df.shape)

# Initial look at the data
print("\n--- Train DataFrame Info ---")
print(train_df.info())

print("\n--- Missing Values Summary (Train) ---")
print(train_df.isnull().sum())

Train Data Shape: (891, 12)
Test Data Shape: (418, 11)

--- Train DataFrame Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None

--- Missing Values Summary (Train) ---
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177


## 2. Data Cleaning and Preparation
*Goal: Address missing values and prepare features for modeling.*

### 2.1 Concatenating Data

In [5]:
# 1. Track the split point
n_train = train_df.shape[0]

# 2. Drop the target variable from the training data
y_train = train_df['Survived']
X_train = train_df.drop('Survived', axis=1)

# 3. Combine the DataFrames
combined_df = pd.concat([X_train, test_df], ignore_index=True)

print(f"Original Training Size (n_train): {n_train}")
print(f"Combined DataFrame Shape: {combined_df.shape}")
print("\n--- Combined DataFrame Info ---")
combined_df.info()

Original Training Size (n_train): 891
Combined DataFrame Shape: (1309, 11)

--- Combined DataFrame Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Pclass       1309 non-null   int64  
 2   Name         1309 non-null   object 
 3   Sex          1309 non-null   object 
 4   Age          1046 non-null   float64
 5   SibSp        1309 non-null   int64  
 6   Parch        1309 non-null   int64  
 7   Ticket       1309 non-null   object 
 8   Fare         1308 non-null   float64
 9   Cabin        295 non-null    object 
 10  Embarked     1307 non-null   object 
dtypes: float64(2), int64(4), object(5)
memory usage: 112.6+ KB


### 2.2 Handling Missing Data

In [14]:
# Impute the missing data in the 'Fare' column with the median
median_fare = combined_df['Fare'].median()
combined_df.fillna({'Fare': median_fare}, inplace=True)
combined_df['Fare'].isna().sum()

np.int64(0)

In [20]:
# Impute the missing data in the 'Embarked' column with the mode
mode_embarked = combined_df['Embarked'].mode()[0]
combined_df.fillna({'Embarked': mode_embarked}, inplace=True)
combined_df['Embarked'].isna().sum()

np.int64(0)

In [26]:
# Impute the missing data in the 'Age' column with the median grouped by 'Pclass' and 'Sex'
grouped_median_age = combined_df.groupby(['Pclass', 'Sex'])['Age'].transform('median')
combined_df['Age'] = combined_df['Age'].fillna(grouped_median_age)
combined_df['Age'].isna().sum()

np.int64(0)

In [30]:
# Drop the 'Cabin' column since most of the values are missing
combined_df.drop('Cabin', axis=1, inplace=True)

In [31]:
combined_df.isna().sum()

PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64

### 2.3 Feature Engineering

In [36]:
# Extract titles from names
combined_df['Title'] = combined_df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)

# Map titles for simplification
title_mapping = {
    "Mlle": "Miss",
    "Major": "Other",
    "Col": "Other",
    "Sir": "Other",
    "Don": "Other",
    "Mme": "Mrs",
    "Lady": "Other",
    "Capt": "Other",
    "Countess": "Other",
    "Jonkheer": "Other",
    "Dona": "Other",
    "Ms": "Miss",
    "Dr": "Other",
    "Rev": "Other",
    "Master": "Master",
    "Mr": "Mr",
    "Mrs": "Mrs",
    "Miss": "Miss",
}

combined_df['Title'] = combined_df['Title'].map(title_mapping)
combined_df['Title'].value_counts()

<>:2: SyntaxWarning: invalid escape sequence '\.'
<>:2: SyntaxWarning: invalid escape sequence '\.'
C:\Users\user\AppData\Local\Temp\ipykernel_18436\1321929244.py:2: SyntaxWarning: invalid escape sequence '\.'
  combined_df['Title'] = combined_df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)


Title
Mr        757
Miss      264
Mrs       198
Master     61
Other      29
Name: count, dtype: int64

- `FamilySize`: Create the total number of people in a family unit.

In [37]:
combined_df['FamilySize'] = combined_df['SibSp'] + combined_df['Parch'] + 1

- `IsAlone`: Create a binary flag for people traveling alone.

In [38]:
combined_df['IsAlone'] = 0
combined_df.loc[combined_df['FamilySize'] == 1, 'IsAlone'] = 1

- Drop Irrelevant/Engineered Columns

In [40]:
combined_df.drop(['Parch', 'SibSp', 'Name', 'Ticket'], axis=1)

,PassengerId,Pclass,Sex,Age,Fare,Embarked,Title,FamilySize,IsAlone
0,1,3,male,22.0,7.2500,S,Mr,2,0
1,2,1,female,38.0,71.2833,C,Mrs,2,0
2,3,3,female,26.0,7.9250,S,Miss,1,1
3,4,1,female,35.0,53.1000,S,Mrs,2,0
4,5,3,male,35.0,8.0500,S,Mr,1,1
...,...,...,...,...,...,...,...,...,...
1304,1305,3,male,25.0,8.0500,S,Mr,1,1
1305,1306,1,female,39.0,108.9000,C,Other,1,1
1306,1307,3,male,38.5,7.2500,S,Mr,1,1
1307,1308,3,male,25.0,8.0500,S,Mr,1,1
